In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from main import *
from v_p_chapa_cabeca import *
from materiais import *

In [2]:
perfil_escolhido = W_250x32_7

aço_escolhido = ASTM_A36

solda_escolhida =E70XX

In [3]:
chapa = chapa_para_perfil(perfil_escolhido)

In [4]:
disp_parafuso = disposicao_parafusos(perfil_escolhido)
ver_parafuso = disp_parafuso.df

In [5]:
M = 136000 #kN.mm 

V = 196 #kN

T = 00 #kN

B = chapa.B

parafuso = ASTM_A325

diametros = diametros_A325

rosca = 1

planos_de_corte =1

N = len(ver_parafuso)  #Número total de parafusos

n = (ver_parafuso["x (mm)"] == ver_parafuso["x (mm)"].iloc[0]).sum()  #número de parafusos por coluna

n_p_c = N/n  #número de parafusos por camada

S = curva_interacao(M,V,T,B,np.unique(ver_parafuso["y (mm)"]),parafuso,rosca,planos_de_corte,n,n_p_c,diametros,gamma_a2)

print(S)

   k diametro       y_ln
0  0    1.1/8  75.894495


In [6]:
rigida = 1

F_r_total = resistencia_total(parafuso,rosca,planos_de_corte,gamma_a2)

F_t_Sd=solicitante_parafuso_tração(M,B, np.unique(ver_parafuso["y (mm)"]), parafuso , n, S.k[0]) + T

exp = exp_placa(aço_escolhido, chapa, espessuras, rigida, ver_parafuso, pol_to_mm(S.diametro[0]), F_r_total,F_t_Sd,gamma_a1)

pol_to_mm(exp)

25.4

In [7]:
esp_solda(aço_escolhido,solda_escolhida,(perfil_escolhido.h-2*perfil_escolhido.t_f), M,V,T,True,gamma_a1,gamma_a2)

np.float64(23.42779191199343)

In [8]:
import pandas as pd
from pyautocad import Autocad, APoint

def criar_desenho_autocad(df, nome_arquivo="chapa_aco.dwg"):
    """
    Cria um arquivo de AutoCAD com a seção da chapa de aço a partir de um DataFrame.

    Args:
        df (pd.DataFrame): DataFrame com as coordenadas dos vértices.
                           Deve conter colunas 'x (mm)' e 'y (mm)' com as coordenadas em mm.
        nome_arquivo (str): Nome do arquivo AutoCAD a ser criado (com extensão .dwg).
    """
    try:
        acad = Autocad(create_if_not_exists=True)
        print(f"Conectado ao AutoCAD versão: {acad.Application.Version}")
    except Exception as e:
        print(f"Erro ao conectar ao AutoCAD: {e}")
        return

    msp = acad.model  # Acessa o Model Space

    # Prepara a lista de coordenadas para a Polyline
    coords =[]
    for index, row in df.iterrows():
        coords.extend([row['x (mm)'], row['y (mm)'], 0])  # Adiciona a coordenada Z (assumindo 2D)

    # Cria a Polyline
    if coords:
        ponto_inicial = APoint(coords[0], coords[1], coords[2])
        polyline = msp.AddPolyline(coords)

        # Fecha a Polyline (se a chapa for fechada)
        if not df.empty and (df.iloc[0]['x (mm)'] != df.iloc[-1]['x (mm)'] or df.iloc[0]['y (mm)'] != df.iloc[-1]['y (mm)']):
            polyline.Closed = True

        # Salva o desenho
        try:
            acad.doc.SaveAs(nome_arquivo)
            print(f"Arquivo AutoCAD '{nome_arquivo}' criado com sucesso.")
        except Exception as e:
            print(f"Erro ao salvar o arquivo AutoCAD: {e}")
    else:
        print("O DataFrame não contém coordenadas para criar a seção.")

    criar_desenho_autocad(chapa.df, nome_arquivo="chapa_aberta.dwg")

ModuleNotFoundError: No module named 'pyautocad'